# Using pre-trained CNN

Usare modelli già addestrati da altri

In this lab, we will see:

- Zero-shot performance of pre-trained backbone, perfomance del modello senza fare nulla, si prende un modello e si testa. Se non addestro il modello per nulla sui dati, darò poche performance
- Use pre-trained CNN as backbone, utilizzarlo come backbone del modello, spesso un modello viene suddiviso in due parti: una backbone (corpo, che può essere CNN, trasformer...) e un classifier (head). Si va ad addestrare solo la testa con un modello pre-trained (la testa sarà inizializzata random), quando si hanno pochi dati (uso modello già pre-trained), avrò performance buone ma non quanto quelle di finetuning. Qui la backbone è frozen. Se addestro solo la testa, avrò performance migliori rispetto a prima
- Fine-tuning the pre-trained CNN, aggiorna anche i pesi della backbone, si spostano anche i pesi della CNN, la testa si adatta sui dati che abbiamo e anche i pesi della rete. Questo porta beneficio. Non ci dimentichiamo del passato, non dobbiamo sciupare performance modelli pre-trained. I modelli pretrained sono stati addestrati su dataset molto grossi, con un numero di immagini molto elevato rispetto a quelle che abbiamo in locale. Vogliamo adattare modello per apprendere i dati che abbiamo, ma senza sciupare più di tanto i dati che conosceva e su cui era stato appreso. Se sposto un pochino anche la mia backbone, avrò performance migliori

(punti ordinati in base alla facilità di implementazione e alle performance)


!!PARAGONE CIFAR18 RESNET VS QUESTA RETE (3 punti)!! pre-trained model funziona meglio

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 10
device = torch.device("cuda") # to use the GPU

In [ ]:
# usiamo sempre CIFAR10, import dei dati
# quello che vediamo è usare dei modelli pre-trained model su ImageNet e lo lanciamo su CIFAR10 e vediamo quanto migliorano le performance
# con CIFAR10 dell'altra volta si riusciva ad arrivare a delle performance circa del 60%, vediamo se con queste tre opzioni riusciamo a migliorare (sì)

# RandomCrop la croppo 32x32 in modo randomico, prima di fare il crop viene fatto il padding con dei valori neri (quindi a 0), l'immagine diventa 36 x 36 (perché CIFAR10 di partenza è 32 x 32)
# quindi l'immagine sarà un po' spostata
# RandomHorizontalFlip, prob = 1/2  (0.5) di fare flipping (di base), rovescia a speccio

# quando si fa il training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

# create a split for train/validation. We can use early stop, utile per questo, ovvero vuol dire scegliere il modello migliore che vogliamo portarci dietro sulla base del validation set
# l'idea è questa: quando si fa un training molto lungo con tante epoche, ci sarà un certo momento in cui avrò performance migliori e poi andando avanti si andrà a peggiorare
# quindi quando avrò ottenuto la performance migliore voglio salvarmi quello stato e poi utilizzarlo per fare il test. Questo è Early Stopping, ma non dobbiamo usare il test set, ma il val set
# il test set va usato solo a fine apprendimento, sennò è barare!!


trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)


Files already downloaded and verified
Files already downloaded and verified


## Load a pre-defined network with pretrained weights
come si prendono le reti pre-addestrate da pytorch?



In [ ]:
# dalla libreria torchvision.models ci sono definite tutte le architetture
# si definiscono tutte come sotto, prendono pretrained in ingresso se è = TRUE ci dà modello con i pesi pre-addestrati su ImageNet

net = models.resnet18(pretrained=True)
net

# PROBLEMA: RETE STESSA!
# problema: se io apprendo dei pesi sulle immagini 24 x 24 (imagenet), le posso usare su immagini su 32 x 32 (CIFAR)
# problema: imagenet ha 1000 classi (out_features), CIFAR10 ne ha 10, quello che viene fatto è buttare via l'ultimo livello della rete (fc, cioè il classifier) e andare a rinizializzarlo con un classificatore con 10 classi

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 40.6MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:


net = models.resnet18(pretrained=True) # pretrained = True ci inizializza il modello con i pesi pre-addestrati su imagenet


# per sovrascrivere il classificatore quello che si fa si fa un override e si fa uscire con una dimensione = 10

# override the fc layer of the network since it is of 1000 classes by default (ImageNet)
net.fc = nn.Linear(512, 10) # si fa override layer fc, si fa uscire con una uscita a 10 invece che a 1000, il fc così però avrà dei valori randomici, si vede da "out_features"
# quindi si perdere il classificatore addestrato su ImageNet, ma abbiamo a questo punto una rete pre-addestrata su ImageNet con l'unica eccezione che il fc finale avrà pesi randomici

net.to(device)

# 2048 resnet50 al posto di 512

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

ogni rete può essere presa nei suoi layer intermedi attraverso il .

In [ ]:
# posso accedere a qualsiasi livello

net.conv1

#e ai pesi

#net.conv1.weight

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

la ResNet è costruita a blocchi, layer1,2,3,4

Ogni singolo layer è un modulo che si chiama Sequential perché a sua volta è formato da dei blocchi che sono il blocco 0 e il blocco 1

I blocchi Sequential sono una lista, si accedono così:

In [ ]:
net.layer1

#net.layer1[0] # accedo al blocco

#net.layer1[0].conv1

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

Nella ResNet ci sono sia dei layer che dei blocchi sequenziali. I blocchi sequenziali sono questi che chiamano layer1,2,3,4 mentre i layer normali sono quelli che hanno come keyword avgpooling, fc, conv, bn, relu e maxpool (dal basso verso l'alto)

In [ ]:
# count the trainable parameters of the model
# conta il numero di parametri che sono addestrabili, comodo per vedere quanti layer sto addestrando e quanti li sto freezando


def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) # requires_grad mi dice se è freezato o learnable, TRUE deve essere appreso, FALSE no, ritorna la somma dei parametri che richiedono gradiente
count_trainable_parameters(net)

# la ResNet18 ha 11 milioni di parametri (11181642)

0

vogliamo agire sui paramtetri del modello, ne fissiamo alcuni mentre learnable degli altri

In [ ]:
# frozen all the weights of the network, except for fc ones
# fissiamo alcuni e altri li mettiamo learnable
for param in net.parameters():
    param.requires_grad = False # mettiamo a false TUTTI i parametri della rete # se non facessimo altro la rete sarebbe non apprendibile
#net.fc.weight.requires_grad = True # metto i pesi dei rg a true nel fc, quaindi al classificatore, sia al peso che al bias # FROZEN
#net.fc.bias.requires_grad = True # e anche al bias del classificatore, l'ultimo layer prende un input y = x*W^T + b, input x matrice dei pesi + bias, teniamo freezati tutti i pesi tranne W e b # FROZEN
#count_trainable_parameters(net)

# ho un modello grosso di 11 milioni di parametri, ma ne addestro solo 5000

quindi: ho fatto override sul classificatore inizializzato a random, voglio addestrarlo sui miei (pochi) dati può essere funzionale perché ho un modello grosso con 11 milioni di parametri, ma di questi ne addestro solo 5000

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-04)

In [ ]:
# the main loop
train_losses = []
val_losses = []
val_accuracies = []
model_state_dict = None

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

# perché la loss sta andando ad aumentare?
# la backbone è completamente diversa da quella che è la testa, la testa si spostando dai dati che abbiamo di partenza, non abbiamo grosse performance con lr=0.1
# no grande performance, migliori con 31%



# vorrei loss di training (bassa), accuratezza di validation (alta), monitorare una delle due

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.584742
Train Epoch: 1 [32000/40000 (80%)]	Loss: 2.453959
\Test set: Average loss: 412.9323, Accuracy: 984/10000 (10%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 2.749192
Train Epoch: 2 [32000/40000 (80%)]	Loss: 2.496544
\Test set: Average loss: 412.3418, Accuracy: 952/10000 (10%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 2.547477
Train Epoch: 3 [32000/40000 (80%)]	Loss: 2.505162
\Test set: Average loss: 407.9046, Accuracy: 971/10000 (10%)

Train Epoch: 4 [0/40000 (0%)]	Loss: 2.740646
Train Epoch: 4 [32000/40000 (80%)]	Loss: 2.701255
\Test set: Average loss: 409.3044, Accuracy: 989/10000 (10%)

Train Epoch: 5 [0/40000 (0%)]	Loss: 2.583096
Train Epoch: 5 [32000/40000 (80%)]	Loss: 2.701384
\Test set: Average loss: 411.3036, Accuracy: 939/10000 (9%)

Train Epoch: 6 [0/40000 (0%)]	Loss: 2.594532
Train Epoch: 6 [32000/40000 (80%)]	Loss: 2.651295
\Test set: Average loss: 414.4543, Accuracy: 951/10000 (10%)

Train Epoch: 7 [0/40000 (0%)]	Loss: 2.501345
Train Epoch: 7

In [ ]:
# fare il load dei pesi sulla rete # QUI

test_loss, test_acc = test(net, device, testloader)

\Test set: Average loss: 340.5006, Accuracy: 3551/10000 (36%)



zero-shot: non eseguire cella grad, 15%, freezo tutto, prova train (non fa) rieseguo cella overraid e poi faccio train (fa)

2: abbiamo freezato tutti i pesi della backbone e tenere soltanto i pesi del classificatore liberi, la loss parte bassa ma aumenta di molto

## Add additional layer to the pre-trained model


In [ ]:
fc1 = nn.Linear(512, 128)

# 512 era l'uscita resnet 128

# Modify the existing fully connected layer (fc)
net.fc = nn.Linear(128, 10)

# poi ci metto 10

# Replace the model's classifier with a new sequential layer
# that includes the new fc1 and the modified fc
net.fc = nn.Sequential(
    fc1,
    nn.ReLU(),   # Optional: Add an activation function like ReLU
    net.fc # classificatore finale
)
net.to(device)



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Fine-tuning some part of the CNN (not only the classifier)

non spostiamo solo il fc, ma spostiamo anche qualche layer precedente

ma quanti layer finetunare? quanti dei 4 layer voglio spostare? tutti? uno? due? tre?

In [ ]:
# quanti layer andare a fine-tunare? quanti di questi layer io voglio spostare, aggiornare?
# qui spostiamo solo il 4° blocco della rete, lasciando freezed 1 2 3
# oltre a sbloccare il fc, il classificare, mettiamo a True i parametri del layer 4

# perchè si fa così?

# i layer di una rete convoluzionale sono tipicamente più o meno sempre gli stessi, imparano filtri simili, man mano che andiamo in profondità con i layer, i filtri diventano sempre più specifici
# spesso si finetuna l'ultimo layer e poi quelli un pochino precedenti, però sempre verso la cima della rete
# per far questo, una cosa che si può fare è andare a specializzare dei vari valori di lr sulla base di quale layer noi stiamo calcolando lo spostamento del gradiente


for param in net.parameters():
    param.requires_grad = False

# Unfreeze layer4 parameters, all'interno 4° blocco della rete
for param in net.layer4.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

print(count_trainable_parameters(net))

8393728


In [ ]:
# Setting different learning rates, si vanno a specilizzare vari valori di lr in base a dove stiamo calcolando lo spostamento del gradiente
# specializziamo il lr in base se stiamo aggiornando il classificatore o layer convoluzionali
# è bene spostarli, ma di poco
# bisogna spostare poco i layer che sono già stati pre-addestrati, ecco perchè spostatiamo poco il layer 4
# mentre dobbiamo spostare tanto, in maniera corretta, quei layer che sono stati inizializzati a random, per esempio il classificatore

# lay3
layer4_params = {'params': net.layer4.parameters(), 'lr': 0.001} # circa 7995000
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer4_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728
# posso specializzare quali sono i suoi parametri, questo vuol dire che questi parametri del layer 4 verranno aggiornati con un lr =0.001 con un momentum 0.9 e con quel weight_decay
# mentre i parametri del fc avrabbi lr = 0.1 momentum 0.9 e quel weight_decay

# così si definiscono diversi lr per diversi layer


#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-04) # invece di passargli questo...



# vediamo cosa succede al training

qui su 11 milioni di parametri ne sto spostando 8 milioni! (layer 4), è vero che stiamo spostando poco ma stiamo spostando molti paramentri

## Exercise 1

How many layers it is better to fine-tune?

It is better to update all the weights of the model?

andare a lavorare sul numero di layer di finetunare, si possono fare anche tutti, è bene spostarli un pochino tutti, risultati migliori quando li sposto tutti (risultati migliori), da pochi a tanti

noi abbiamo finetunato tutto il layer 4, magari mettere learnable blocco 0 e freezed 1

quaesti modelli più grandi sono, più la convergenza è lenta

scegliere quanti livelli finetunare. vedere da pochi a molti livelli cosa succede, e sulla base di questo, fare esercizio 2, cercare di aumentare l'accuracy il più possibile sul test

# LAYER 4 + FC

In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 10
device = torch.device("cuda") # to use the GPU

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 10)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for param in net.parameters():
    param.requires_grad = False

# Unfreeze layer4 parameters, all'interno 4° blocco della rete
for param in net.layer4.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

print(count_trainable_parameters(net))

8393728


In [ ]:
layer4_params = {'params': net.layer4.parameters(), 'lr': 0.001} # circa 7995000
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer4_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728

criterion = nn.CrossEntropyLoss()

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
model_state_dict = None

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.614350
Train Epoch: 1 [32000/40000 (80%)]	Loss: 1.438805
\Test set: Average loss: 205.7057, Accuracy: 5416/10000 (54%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 1.541163
Train Epoch: 2 [32000/40000 (80%)]	Loss: 1.067788
\Test set: Average loss: 186.1315, Accuracy: 5801/10000 (58%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 1.209922
Train Epoch: 3 [32000/40000 (80%)]	Loss: 1.176043
\Test set: Average loss: 178.8759, Accuracy: 5983/10000 (60%)

Train Epoch: 4 [0/40000 (0%)]	Loss: 1.067012
Train Epoch: 4 [32000/40000 (80%)]	Loss: 0.883793
\Test set: Average loss: 170.4402, Accuracy: 6162/10000 (62%)

Train Epoch: 5 [0/40000 (0%)]	Loss: 0.952084
Train Epoch: 5 [32000/40000 (80%)]	Loss: 1.138575
\Test set: Average loss: 167.1979, Accuracy: 6249/10000 (62%)

Train Epoch: 6 [0/40000 (0%)]	Loss: 1.121830
Train Epoch: 6 [32000/40000 (80%)]	Loss: 1.086853
\Test set: Average loss: 164.7648, Accuracy: 6331/10000 (63%)

Train Epoch: 7 [0/40000 (0%)]	Loss: 0.982751
Train E

In [ ]:
# fare il load dei pesi sulla rete # QUI

test_loss, test_acc = test(net, device, testloader)

\Test set: Average loss: 154.3832, Accuracy: 6598/10000 (66%)



# LAYER 1 + FC

In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 10
device = torch.device("cuda") # to use the GPU

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 10)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for param in net.parameters():
    param.requires_grad = False

# Unfreeze layer1 parameters, all'interno 1° blocco della rete
for param in net.layer1.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

print(count_trainable_parameters(net))

147968


In [ ]:
layer1_params = {'params': net.layer1.parameters(), 'lr': 0.001} # circa 7995000
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer1_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728

criterion = nn.CrossEntropyLoss()

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
model_state_dict = None

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.564060
Train Epoch: 1 [32000/40000 (80%)]	Loss: 2.414378
\Test set: Average loss: 388.4275, Accuracy: 1279/10000 (13%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 2.560171
Train Epoch: 2 [32000/40000 (80%)]	Loss: 2.358895
\Test set: Average loss: 370.7786, Accuracy: 1495/10000 (15%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 2.266311
Train Epoch: 3 [32000/40000 (80%)]	Loss: 2.308924
\Test set: Average loss: 356.6931, Accuracy: 1735/10000 (17%)

Train Epoch: 4 [0/40000 (0%)]	Loss: 2.360715
Train Epoch: 4 [32000/40000 (80%)]	Loss: 2.143639
\Test set: Average loss: 349.4039, Accuracy: 1845/10000 (18%)

Train Epoch: 5 [0/40000 (0%)]	Loss: 2.267616
Train Epoch: 5 [32000/40000 (80%)]	Loss: 2.116818
\Test set: Average loss: 342.0324, Accuracy: 2064/10000 (21%)

Train Epoch: 6 [0/40000 (0%)]	Loss: 2.096284
Train Epoch: 6 [32000/40000 (80%)]	Loss: 2.246629
\Test set: Average loss: 329.6423, Accuracy: 2508/10000 (25%)

Train Epoch: 7 [0/40000 (0%)]	Loss: 2.007871
Train E

In [ ]:
# fare il load dei pesi sulla rete # QUI

test_loss, test_acc = test(net, device, testloader)

\Test set: Average loss: 320.9119, Accuracy: 2828/10000 (28%)



# LAYER 2 + FC

In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 10
device = torch.device("cuda") # to use the GPU

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 10)
net.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for param in net.parameters():
    param.requires_grad = False

# Unfreeze layer2 parameters, all'interno 2° blocco della rete
for param in net.layer2.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

print(count_trainable_parameters(net))

525568


In [ ]:
layer2_params = {'params': net.layer2.parameters(), 'lr': 0.001} # circa 7995000
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer2_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728

criterion = nn.CrossEntropyLoss()

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
model_state_dict = None

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.666121
Train Epoch: 1 [32000/40000 (80%)]	Loss: 2.385042
\Test set: Average loss: 355.3454, Accuracy: 1914/10000 (19%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 2.191776
Train Epoch: 2 [32000/40000 (80%)]	Loss: 2.129533
\Test set: Average loss: 316.3587, Accuracy: 2928/10000 (29%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 2.015850
Train Epoch: 3 [32000/40000 (80%)]	Loss: 1.928674
\Test set: Average loss: 290.7067, Accuracy: 3692/10000 (37%)

Train Epoch: 4 [0/40000 (0%)]	Loss: 1.867131
Train Epoch: 4 [32000/40000 (80%)]	Loss: 1.668746
\Test set: Average loss: 264.4249, Accuracy: 4478/10000 (45%)

Train Epoch: 5 [0/40000 (0%)]	Loss: 1.777490
Train Epoch: 5 [32000/40000 (80%)]	Loss: 1.589396
\Test set: Average loss: 248.4393, Accuracy: 4935/10000 (49%)

Train Epoch: 6 [0/40000 (0%)]	Loss: 1.481752
Train Epoch: 6 [32000/40000 (80%)]	Loss: 1.529028
\Test set: Average loss: 230.0963, Accuracy: 5489/10000 (55%)

Train Epoch: 7 [0/40000 (0%)]	Loss: 1.420316
Train E

In [ ]:
# fare il load dei pesi sulla rete # QUI

test_loss, test_acc = test(net, device, testloader)

\Test set: Average loss: 179.1748, Accuracy: 6697/10000 (67%)



# LAYER 3 + FC


In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 10
device = torch.device("cuda") # to use the GPU

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 10)
net.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for param in net.parameters():
    param.requires_grad = False

# Unfreeze layer3 parameters, all'interno 3° blocco della rete
for param in net.layer3.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

print(count_trainable_parameters(net))

2099712


In [ ]:
layer3_params = {'params': net.layer3.parameters(), 'lr': 0.001} # circa 7995000
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer3_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728

criterion = nn.CrossEntropyLoss()

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
model_state_dict = None

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.658584
Train Epoch: 1 [32000/40000 (80%)]	Loss: 2.011906
\Test set: Average loss: 312.6906, Accuracy: 3004/10000 (30%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 2.132085
Train Epoch: 2 [32000/40000 (80%)]	Loss: 1.796978
\Test set: Average loss: 253.4689, Accuracy: 4860/10000 (49%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 1.617855
Train Epoch: 3 [32000/40000 (80%)]	Loss: 1.316226
\Test set: Average loss: 216.1707, Accuracy: 5799/10000 (58%)

Train Epoch: 4 [0/40000 (0%)]	Loss: 1.577679
Train Epoch: 4 [32000/40000 (80%)]	Loss: 1.441144
\Test set: Average loss: 192.5191, Accuracy: 6317/10000 (63%)

Train Epoch: 5 [0/40000 (0%)]	Loss: 1.253528
Train Epoch: 5 [32000/40000 (80%)]	Loss: 1.241703
\Test set: Average loss: 175.9707, Accuracy: 6622/10000 (66%)

Train Epoch: 6 [0/40000 (0%)]	Loss: 1.003162
Train Epoch: 6 [32000/40000 (80%)]	Loss: 1.272495
\Test set: Average loss: 166.5666, Accuracy: 6809/10000 (68%)

Train Epoch: 7 [0/40000 (0%)]	Loss: 1.065634
Train E

In [ ]:
# fare il load dei pesi sulla rete # QUI

test_loss, test_acc = test(net, device, testloader)

\Test set: Average loss: 136.2675, Accuracy: 7373/10000 (74%)



## Exercise 2

Try to change the hyper-parameters of the fine-tuning (e.g. lr of CNN layers and lr of the fc layers) and/or network architecture

# LAYER 1-4 + FC con lr = 0.001

In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 10
device = torch.device("cuda") # to use the GPU


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 10)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) # requires_grad mi dice se è freezato o learnable, TRUE deve essere appreso, FALSE no, ritorna la somma dei parametri che richiedono gradiente
count_trainable_parameters(net)

for param in net.parameters():
    param.requires_grad = False

#Unfreeze layer1 parameters, all'interno 1° blocco della rete
for param in net.layer1.parameters():
    param.requires_grad = True
for param in net.layer2.parameters():
    param.requires_grad = True
for param in net.layer3.parameters():
    param.requires_grad = True
for param in net.layer4.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

#net.fc.weight.requires_grad = True
#net.fc.bias.requires_grad = True

print(count_trainable_parameters(net))

11166976


In [ ]:
layer1_params = {'params': net.layer1.parameters(), 'lr': 0.001}
layer2_params = {'params': net.layer2.parameters(), 'lr': 0.001}
layer3_params = {'params': net.layer3.parameters(), 'lr': 0.001}
layer4_params = {'params': net.layer4.parameters(), 'lr': 0.001}
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer1_params, layer2_params, layer3_params, layer4_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728

criterion = nn.CrossEntropyLoss()

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
model_state_dict = None

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.617908
Train Epoch: 1 [32000/40000 (80%)]	Loss: 1.077149
\Test set: Average loss: 144.7487, Accuracy: 6812/10000 (68%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 0.738457
Train Epoch: 2 [32000/40000 (80%)]	Loss: 1.102464
\Test set: Average loss: 122.3912, Accuracy: 7349/10000 (73%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 0.673748
Train Epoch: 3 [32000/40000 (80%)]	Loss: 0.850887
\Test set: Average loss: 110.5737, Accuracy: 7529/10000 (75%)

Train Epoch: 4 [0/40000 (0%)]	Loss: 0.760127
Train Epoch: 4 [32000/40000 (80%)]	Loss: 0.629086
\Test set: Average loss: 104.7687, Accuracy: 7672/10000 (77%)

Train Epoch: 5 [0/40000 (0%)]	Loss: 0.662762
Train Epoch: 5 [32000/40000 (80%)]	Loss: 0.574579
\Test set: Average loss: 102.8108, Accuracy: 7719/10000 (77%)

Train Epoch: 6 [0/40000 (0%)]	Loss: 0.661864
Train Epoch: 6 [32000/40000 (80%)]	Loss: 0.678284
\Test set: Average loss: 96.7023, Accuracy: 7836/10000 (78%)

Train Epoch: 7 [0/40000 (0%)]	Loss: 0.852176
Train Ep

In [ ]:
# fare il load dei pesi sulla rete # QUI

test_loss, test_acc = test(net, device, testloader)

\Test set: Average loss: 84.0407, Accuracy: 8188/10000 (82%)



# LAYER 1-4 + FC con lr = 0.0001 (1-4)

In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 10
device = torch.device("cpu") # to use the GPU

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 10)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) # requires_grad mi dice se è freezato o learnable, TRUE deve essere appreso, FALSE no, ritorna la somma dei parametri che richiedono gradiente
count_trainable_parameters(net)

for param in net.parameters():
    param.requires_grad = False

#Unfreeze layer1 parameters, all'interno 1° blocco della rete
for param in net.layer1.parameters():
    param.requires_grad = True
for param in net.layer2.parameters():
    param.requires_grad = True
for param in net.layer3.parameters():
    param.requires_grad = True
for param in net.layer4.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

#net.fc.weight.requires_grad = True
#net.fc.bias.requires_grad = True

print(count_trainable_parameters(net))

11166976


In [ ]:
layer1_params = {'params': net.layer1.parameters(), 'lr': 0.0001}
layer2_params = {'params': net.layer2.parameters(), 'lr': 0.0001}
layer3_params = {'params': net.layer3.parameters(), 'lr': 0.0001}
layer4_params = {'params': net.layer4.parameters(), 'lr': 0.0001}
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer1_params, layer2_params, layer3_params, layer4_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728

criterion = nn.CrossEntropyLoss()

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
model_state_dict = None

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.580923
Train Epoch: 1 [32000/40000 (80%)]	Loss: 1.802090
\Test set: Average loss: 244.1810, Accuracy: 4660/10000 (47%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 1.232132


KeyboardInterrupt: ignored

In [ ]:
# fare il load dei pesi sulla rete # QUI

test_loss, test_acc = test(net, device, testloader)

## Exercise 3

Try to implement the model selection strategy (also known as early stopping) based on the validation accuracy on cifar10.

Consider using the two following command to respectively save and load the state of all the parameters of the model in a moment.

vorrei cercare il modello con più alta performance, prendere stato parametri e salvarmela da un'altra parte

implementare una strategia di model selection (quello che si diceva nell'early stopping): all'interno delle mie 10 epoche, io vorrei andare a cercare qual è il modello che ha la più alta performance, all'interno di queste epoche, e prendere lo stato dei suoi parametri e salvarmelo da una parte. Poi questa parte io la prendo e, alla fine del mio training, nel mio setting migliore, lo voglio ri-inizializzare sul modello e usare quello a test time.

In [ ]:
# save all the parameters of the model
model_state_dict = net.state_dict()
# ritorna il dizionario dei parametri. lo saliamo quando abbiamo la migliore accuratezza sul validation set
# in model_state_dict avrò il dizionario con tutti i layer della rete con tutti i valori dei parametri associati a quei layer
# questo dizionario è la rete, la rete la fanno i pesi
# si salva quando ho la migliore accuratezza sul validation set


# nel momento in cui è finito il training, ad es 100 epoche, se la best accuracy l'ho trovata alla epoca 60, ricarichiamo i pesi dell'epoca 60 nel modello per poi usarla a test time


# load saved weights on the model
net.load_state_dict(model_state_dict)
# ricarico il modello, carica un dizionario di parametri del modello, sovrascrive tutti parametri della rete, gli passiamo lo state_dict dell'epoca migliore


# modificare il loop di training, prima di chiamare la funzione che fa il test (#QUI) facciamo il load dello state dict sulla rete per ricaricare i pesi migliori

<All keys matched successfully>

# SAVE MODEL & LOAD MODEL

In [ ]:
batch_size = 64
lr = 0.01 # 0.01  #0.1
epochs = 20
device = torch.device("cuda") # to use the GPU


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# non faccio nessuna trasformazione a test time
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000]) # dataset, numero immagini train 40000 e validation 10000, in uscita avrò due dataset


# costruisco i dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,
                                          drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2,
                                          drop_last=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2,
                                          drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 10)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) # requires_grad mi dice se è freezato o learnable, TRUE deve essere appreso, FALSE no, ritorna la somma dei parametri che richiedono gradiente
count_trainable_parameters(net)

for param in net.parameters():
    param.requires_grad = False

#Unfreeze layer1 parameters, all'interno 1° blocco della rete
for param in net.layer1.parameters():
    param.requires_grad = True
for param in net.layer2.parameters():
    param.requires_grad = True
for param in net.layer3.parameters():
    param.requires_grad = True
for param in net.layer4.parameters():
    param.requires_grad = True

# Unfreeze fc layer parameters
net.fc.requires_grad = True

#net.fc.weight.requires_grad = True
#net.fc.bias.requires_grad = True

print(count_trainable_parameters(net))

11166976


In [ ]:
layer1_params = {'params': net.layer1.parameters(), 'lr': 0.001}
layer2_params = {'params': net.layer2.parameters(), 'lr': 0.001}
layer3_params = {'params': net.layer3.parameters(), 'lr': 0.001}
layer4_params = {'params': net.layer4.parameters(), 'lr': 0.001}
fc_params = {'params': net.fc.parameters(), 'lr': 0.1} # circa 5000
# bisogna spostare poco quei layer che sono già stati addestrati, invce vanno spostati in maniera corretta i random


# come si fa ad inizializzare un ottimizzatore pwe agire su questi parametri?
# Assuming you are using an Adam optimizer
optimizer = torch.optim.SGD([layer1_params, layer2_params, layer3_params, layer4_params, fc_params], momentum=0.9, weight_decay=1e-04) # 8393728

criterion = nn.CrossEntropyLoss()

In [ ]:
# define train and test function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())
    return np.mean(losses)

def test(model, device, test_loader, val=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    mode = "Val" if val else "Test"
    print('\{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        mode,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = correct / len(test_loader.dataset)
    return test_loss, test_acc

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
current_val_loss = np.inf

for epoch in range(1, epochs + 1):
    train_loss = train(net, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    val_loss, val_acc = test(net, device, valloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    if val_loss <= current_val_loss:
      current_val_loss = val_loss
      torch.save(net.state_dict(), f'model_parameters.pth')
    else:
      break




Train Epoch: 1 [0/40000 (0%)]	Loss: 2.627929
Train Epoch: 1 [32000/40000 (80%)]	Loss: 0.918371
\Test set: Average loss: 144.5766, Accuracy: 6804/10000 (68%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 0.900742
Train Epoch: 2 [32000/40000 (80%)]	Loss: 0.562562
\Test set: Average loss: 123.2627, Accuracy: 7267/10000 (73%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 0.806990
Train Epoch: 3 [32000/40000 (80%)]	Loss: 0.679145
\Test set: Average loss: 111.6143, Accuracy: 7541/10000 (75%)

Train Epoch: 4 [0/40000 (0%)]	Loss: 0.490635
Train Epoch: 4 [32000/40000 (80%)]	Loss: 0.603256
\Test set: Average loss: 104.6539, Accuracy: 7675/10000 (77%)

Train Epoch: 5 [0/40000 (0%)]	Loss: 0.647885
Train Epoch: 5 [32000/40000 (80%)]	Loss: 0.811978
\Test set: Average loss: 100.0111, Accuracy: 7824/10000 (78%)

Train Epoch: 6 [0/40000 (0%)]	Loss: 0.966464
Train Epoch: 6 [32000/40000 (80%)]	Loss: 0.655296
\Test set: Average loss: 96.5366, Accuracy: 7904/10000 (79%)

Train Epoch: 7 [0/40000 (0%)]	Loss: 0.758326
Train Ep

In [ ]:
# load saved weights on the model
net.load_state_dict(torch.load(f'model_parameters.pth'))

test_loss, test_acc = test(net, device, testloader)

\Test set: Average loss: 83.3539, Accuracy: 8219/10000 (82%)

